# Assignment2 for RT2 version 2

## Importing the required packages

In [1]:
import matplotlib.pyplot as plt
import rospy
import ipywidgets as widgets
from ipywidgets import Button, Layout, ButtonStyle, VBox, HBox, Label
from assignment_2_2023.msg import CustomStatus
from assignment_2_2023.srv import Input, Goal
from turtlesim.msg import Pose
from matplotlib.animation import FuncAnimation
import threading
import time

## Defining the class with all the required functionalities

In [2]:
class Visualiser:
    def _setup_world(self):
        # box border
        self.ax.plot([9.5,-9.5,-9.5,9.5,9.5],[9.5,9.5,-9.5,-9.5,9.5],'y')
        # obstacles
        self.ax.plot([4, -5, -5], [-0.5, -0.5, -5], 'brown')
        self.ax.plot([-7,5], [4.5,4.5], 'brown')
        
    def _set_goal_status(self):
        counter = 0
        while(True):
            goal = self.get_goal()
            output_display.value = goal.status
            time.sleep(1)
    
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.ln, = plt.plot([], [], 'b-')
        self.x_data, self.y_data = [], []
        self.runner_thread = None
        self.send_input = rospy.ServiceProxy('/assignment2_2023/input_string', Input)
        self.get_goal = rospy.ServiceProxy('/assignment_2_2023/last_goal', Goal)


    def plot_init(self):
        self.ax.set_xlim(-10, 10)
        self.ax.set_ylim(-10, 10)
        self.ax.set_title("Assignment display")
        self.ax.set_xlabel("x")
        self.ax.set_ylabel("y")
        self.ax.invert_xaxis()
        self.ax.invert_yaxis()
        self.ax.set_aspect('equal', adjustable='box')
        self._setup_world()
        return self.ln,

    def odom_callback(self, msg):
        self.y_data.append(msg.y)
        self.x_data.append(msg.x)

    def update_plot(self, frame):
        self.ln.set_data(self.x_data, self.y_data)
        return self.ln,
    
    def on_send_goal_pressed(self, a):
        self.send_input("("+str(next_position_x.value)+","+str(next_position_y.value)+")")
        self.ax.plot(next_position_x.value, next_position_y.value, 'go')
    
    def on_cancel_pressed(self, b):
        goal = self.get_goal()
        self.send_input("cancel")
        self.ax.plot(goal.goal_x, goal.goal_y, 'ro')


    def activate_goal_status_reception(self):
        self.runner_thread = threading.Thread(target=self._set_goal_status)
        self.runner_thread.start()

In [3]:
next_position_x = widgets.FloatText(
    value=1,
    description='X:',
    disabled=False,
    layout=Layout(width = '150px', align = "left")
)

next_position_y = widgets.FloatText(
    value=1,
    description='Y:',
    disabled=False,
    layout = Layout(width = '150px')
)

send_goal = Button(
    description='Send Goal',
    style=ButtonStyle(button_color='lightgreen')
)

cancel = Button(
    description='Cancel',
    layout=Layout(width='auto', align="center"),
    style=ButtonStyle(button_color='red')
)

output_display = widgets.Text(description='Status: ', disabled=True)

a = VBox(
    [
        HBox([
                Label("Please enter the next goal -> "),
                next_position_x, 
                next_position_y, 
                send_goal, 
                cancel
            ]),
        output_display,
        Label("Position graph:"),
    ]
)

display(a)

## Starting the ros node and the thread to update the current status of the goal

In [4]:
rospy.wait_for_service('/assignment2_2023/input_string');
rospy.wait_for_service('/assignment_2_2023/last_goal');

%matplotlib notebook
rospy.init_node('Assignment2_RT2')
vis = Visualiser()
send_goal.on_click(vis.on_send_goal_pressed)
cancel.on_click(vis.on_cancel_pressed)
sub = rospy.Subscriber('assignment_2_2023/customStatus', CustomStatus, vis.odom_callback)
ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)
vis.activate_goal_status_reception()
plt.show(block=True)

ROSInitException: time is not initialized. Have you called init_node()?